In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# 🔹 Step 1: Load dataset
df = pd.read_csv("Crop_recommendation.csv")
df.head()


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [3]:

# 🔹 Step 2: Encode target (Crop Name → Numeric)
label_encoder = LabelEncoder()
df["Crop_Label"] = label_encoder.fit_transform(df["label"])


In [4]:


# N,P,K,temperature,humidity,ph,rainfall,label
# 🔹 Step 3: Normalize input features
features = df[["N", "P", "K", "ph", "rainfall", "temperature", "humidity"]]
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)


In [5]:

# 🔹 Step 4: Prepare data
X = features_scaled
y = to_categorical(df["Crop_Label"])  # One-hot encoding for classification


In [6]:

# 🔹 Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:

# 🔹 Step 6: Build ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y.shape[1], activation='softmax')  # Output layer with softmax for classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


d:\jbook\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:

# 🔹 Step 7: Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)


Epoch 1/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0635 - loss: 3.0555 - val_accuracy: 0.3182 - val_loss: 2.8884
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3295 - loss: 2.7785 - val_accuracy: 0.4006 - val_loss: 2.4226
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4987 - loss: 2.2262 - val_accuracy: 0.6222 - val_loss: 1.7980
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6522 - loss: 1.6454 - val_accuracy: 0.7642 - val_loss: 1.3237
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7553 - loss: 1.2274 - val_accuracy: 0.7841 - val_loss: 1.0226
Epoch 6/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8061 - loss: 0.9771 - val_accuracy: 0.7812 - val_loss: 0.8546
Epoch 7/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8508 - loss: 0.7827 - val_accuracy: 0.8636 - val_loss: 0.7065
Epoch 8/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8947 - loss: 0.6542 - val_accuracy: 0.8807 - val_loss:

In [9]:

# 🔹 Step 8: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9461 - loss: 0.1157  
✅ Model Accuracy: 95.23%


In [10]:

# 🔹 Step 9: Save the model
model.save("ann_crop_recommendation.h5")

# Optional: Save label encoder and scaler using joblib or pickle
import joblib
joblib.dump(label_encoder, "crop_label_encoder.pkl")
joblib.dump(scaler, "crop_scaler.pkl")

['crop_scaler.pkl']

In [11]:
# Example: Predict crop based on input features
sample = np.array([[80, 40, 40, 6.5, 100, 25, 70]])  # example input
sample_scaled = scaler.transform(sample)
prediction = model.predict(sample_scaled)
predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Recommended Crop: {predicted_class[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Recommended Crop: jute


d:\jbook\myenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
